In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../results/result.csv')
df = df[df.run >= 3].copy()
df['speed'] = df['sim-cycles'] / df['time']
df['ipc'] = df['instructions'] / df['cycles']
df['MPKI'] = df['cache-misses'] / df['instructions'] * 1000
df

,simulator,benchmark,sim-cycles,run,time,all_data_cache_accesses,cache-misses,cycles,instructions,speed,ipc,MPKI
3,circt-verilator,RocketCore,100000,3,110097,1.072851e+09,35627.0,1.009278e+09,2.411925e+09,0.908290,2.389753,0.014771
4,circt-verilator,RocketCore,100000,4,109150,1.072792e+09,32842.0,1.000347e+09,2.411924e+09,0.916170,2.411089,0.013617
5,circt-verilator,RocketCore,100000,5,109743,1.073052e+09,34330.0,1.001434e+09,2.411924e+09,0.911220,2.408472,0.014233
6,circt-verilator,RocketCore,100000,6,111225,1.073094e+09,38106.0,1.016503e+09,2.411924e+09,0.899078,2.372768,0.015799
7,circt-verilator,RocketCore,100000,7,109855,1.073098e+09,35214.0,1.011470e+09,2.411925e+09,0.910291,2.384573,0.014600
...,...,...,...,...,...,...,...,...,...,...,...,...
685,essent,FMUL,100000,5,4845832,7.862996e+09,489454935.0,2.240890e+10,1.567728e+10,0.020636,0.699601,31.220646
686,essent,FMUL,100000,6,4856386,7.859778e+09,514586477.0,2.244283e+10,1.567728e+10,0.020591,0.698543,32.823702
687,essent,FMUL,100000,7,4881694,7.864132e+09,519583973.0,2.256044e+10,1.567728e+10,0.020485,0.694902,33.142474
688,essent,FMUL,100000,8,4888519,7.863273e+09,521311523.0,2.258329e+10,1.567728e+10,0.020456,0.694199,33.252668


# Speedup

In [3]:
piv_abs = df.pivot_table(values='speed', columns='simulator', index='benchmark', aggfunc='mean')
baseline_name = 'circt-verilator'
baseline = piv_abs[baseline_name].copy()
piv = piv_abs.apply(lambda x: x / baseline)
piv.loc['Average'] = piv.mean()
styled = piv.style
styled.highlight_max(color='green', axis=1)

simulator,circt-verilator,essent,ksim,repcut-2,vcs,verilator
benchmark,,,,,,
Conv2D,1.000000,0.151838,1.678605,1.538074,0.193467,1.075168
FMUL,1.000000,0.114634,2.669249,nan,0.116968,1.029676
FPU,1.000000,0.968527,2.655328,1.049146,0.198790,1.426225
GEMM,1.000000,0.256414,2.316445,1.502364,0.148969,0.775957
Gemmini,1.000000,0.506065,4.732552,0.404299,0.143802,1.454013
RISCVMini,1.000000,0.383502,1.583681,0.969699,0.090967,1.111272
RocketCore,1.000000,0.192514,1.588824,0.234023,0.102557,1.010567
SHA256,1.000000,0.724118,2.047702,0.378460,0.023231,0.982453
SIGMA,1.000000,0.516401,1.973495,0.143509,0.365849,1.134226


# IPC

In [5]:
piv = df.pivot_table(values='ipc', columns='simulator', index='benchmark', aggfunc='mean')
piv.loc['Average'] = piv.mean()
styled = piv.style
styled.highlight_max(color='green', axis=1).format('{:.3f}')

simulator,circt-verilator,essent,ksim,repcut-2,vcs,verilator
benchmark,,,,,,
Conv2D,1.824,1.145,2.961,1.515,2.618,1.870
FMUL,1.842,0.694,2.976,nan,1.050,1.853
FPU,3.661,2.548,3.720,1.492,2.329,2.963
GEMM,1.712,1.831,3.153,1.362,2.691,1.312
Gemmini,2.580,0.510,2.996,0.509,2.270,2.436
RISCVMini,2.049,2.487,3.221,1.208,2.367,2.136
RocketCore,2.398,1.264,3.241,0.823,1.929,2.183
SHA256,3.122,3.480,3.781,0.419,2.573,3.092
SIGMA,3.261,1.365,3.473,0.494,1.968,3.143


# Cycles

In [6]:
def format_scled(c):
  units = ['', ' k', ' M', ' G', ' T', ' E']
  for unit in units:
    if c < 1000:
      return f'{c:.1f}{unit}'
    c /= 1000

In [7]:
piv = df.pivot_table(values='cycles', columns='simulator', index='benchmark', aggfunc='mean')
piv.loc['Average'] = piv.mean()
styled = piv.style
styled.highlight_min(color='green', axis=1).format(format_scled)

simulator,circt-verilator,essent,ksim,repcut-2,vcs,verilator
benchmark,,,,,,
Conv2D,1.7 G,5.7 G,518.3 M,1.1 G,3.6 G,1.6 G
FMUL,5.1 G,22.6 G,958.1 M,None,21.3 G,5.0 G
FPU,895.4 M,463.1 M,170.0 M,867.8 M,1.5 G,633.0 M
GEMM,1.3 G,2.6 G,282.6 M,895.9 M,3.6 G,1.7 G
Gemmini,29.7 G,29.5 G,3.2 G,73.5 G,101.2 G,20.0 G
RISCVMini,371.5 M,480.1 M,117.4 M,385.4 M,1.2 G,337.6 M
RocketCore,1.0 G,2.6 G,315.0 M,4.3 G,4.0 G,995.0 M
SHA256,68.4 M,50.0 M,18.7 M,193.3 M,734.8 M,70.6 M
SIGMA,3.0 G,2.9 G,765.2 M,21.3 G,3.3 G,2.7 G


# Instructions

In [8]:
def format_scled(c):
  units = ['', ' k', ' M', ' G', ' T', ' E']
  for unit in units:
    if c < 1000:
      return f'{c:.1f}{unit}'
    c /= 1000

In [9]:
piv = df.pivot_table(values='instructions', columns='simulator', index='benchmark', aggfunc='mean')
piv.loc['Average'] = piv.mean()
styled = piv.style
styled.highlight_min(color='green', axis=1).format(format_scled)

simulator,circt-verilator,essent,ksim,repcut-2,vcs,verilator
benchmark,,,,,,
Conv2D,3.1 G,6.6 G,1.5 G,1.7 G,9.5 G,3.0 G
FMUL,9.4 G,15.7 G,2.9 G,None,22.3 G,9.2 G
FPU,3.3 G,1.2 G,632.4 M,1.3 G,3.4 G,1.9 G
GEMM,2.3 G,4.7 G,890.7 M,1.2 G,9.7 G,2.2 G
Gemmini,74.4 G,15.0 G,9.3 G,37.3 G,229.6 G,48.2 G
RISCVMini,761.1 M,1.2 G,378.1 M,465.2 M,2.8 G,720.2 M
RocketCore,2.4 G,3.3 G,1.0 G,3.5 G,7.8 G,2.2 G
SHA256,213.3 M,173.8 M,70.9 M,80.9 M,1.9 G,218.3 M
SIGMA,9.8 G,4.0 G,2.7 G,10.5 G,6.5 G,8.3 G


# MPKI

In [10]:
piv = df.pivot_table(values='MPKI', columns='simulator', index='benchmark', aggfunc='mean')
piv.loc['Average'] = piv.mean()
styled = piv.style
styled.highlight_min(color='green', axis=1).format('{:.4f}')

simulator,circt-verilator,essent,ksim,repcut-2,vcs,verilator
benchmark,,,,,,
Conv2D,0.0106,5.2946,0.0144,5.8509,2.0560,0.0123
FMUL,0.0226,32.4007,0.0085,nan,28.2998,0.0109
FPU,0.0102,0.0222,0.0315,2.7725,0.6737,0.0217
GEMM,0.0175,0.7514,0.0212,12.8224,0.5572,0.0346
Gemmini,16.5623,68.8748,2.8041,33.3878,5.4453,9.2756
RISCVMini,0.0357,0.0185,0.0496,6.8039,0.7578,0.0424
RocketCore,0.0146,0.0151,0.0185,1.6467,3.8892,0.0137
SHA256,0.1290,0.1090,0.2623,22.7929,1.0147,0.1269
SIGMA,0.2535,0.3336,0.0088,0.8108,1.2242,0.0068
